In [1]:
import os
import pandas as pd
import mysql.connector
import configparser

In [2]:
# Leer las credenciales del archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

user = config['database']['user']
password = config['database']['password']
host = config['database']['host']
database = config['database']['database']

In [3]:
# Abrir la base de datos: establecer la conexión
cnx = mysql.connector.connect(user=user, password=password, host=host, database=database)

# crear un cursor
cursor = cnx.cursor()

# seleccionar la base de datos
cursor.execute("USE flujo_caja_directo ")

# =====================================================================================================================

# crear tabla de agregación en blanco (para acumular las transacciones de cada item por mes)
cursor.execute("""
    CREATE TABLE tbl_flujo_reporte (
    id_item INT(11) NOT NULL PRIMARY KEY,
    FOREIGN KEY (id_item) REFERENCES tbl_fcdir_items (id_item)
    );
""")

# Esto creará una nueva tabla con una columna id_item del mismo tipo que la columna id_item de la tabla tbl_fcdir_items, y 
# establecerá id_item como clave primaria.

# copiar los registro de la tabla original a la nueva tabla
cursor.execute("""
    INSERT INTO tbl_flujo_reporte (id_item)
    SELECT id_item FROM tbl_fcdir_items
""")

# =====================================================================================================================

# crear tabla de presupuesto en blanco
cursor.execute("""
    CREATE TABLE tbl_flujo_estimado (
    id_item INT(11) NOT NULL PRIMARY KEY,
    FOREIGN KEY (id_item) REFERENCES tbl_fcdir_items (id_item)
    );
""")

# copiar los registro de la tabla original a la nueva tabla
cursor.execute("""
    INSERT INTO tbl_flujo_estimado (id_item)
    SELECT id_item FROM tbl_fcdir_items
""")

# =====================================================================================================================

# Cerrar cursor
cursor.close()

# cerrar la conexión
cnx.commit()
cnx.close()